In [1]:
import numpy as np
import pandas as pd

In [2]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
prices_recommended = [400, 60, 40, 40, 90, 100, 60, 70, 100, 200]

bought_list = [521, 32, 143, 991]
prices_bought = [300, 50, 400, 40]

### Hit rate at k

In [3]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    hit_rate_at_k = (flags.sum() > 0) * 1
    
    return hit_rate_at_k

In [4]:
hit_rate_at_k(recommended_list, bought_list)

1

### Money precision at k

In [5]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    
    recommended_list = recommended_list[:k]
    prices_recommended = prices_recommended[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    money_precision_at_k = (np.inner(flags, prices_recommended)) / prices_recommended.sum()
    
    
    return money_precision_at_k

In [6]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5)

0.6984126984126984

### Recall at k

In [7]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    recall_at_k = flags.sum() / len(bought_list)
    
    return recall_at_k

In [8]:
recall_at_k(recommended_list, bought_list, k=5)

0.5

### Money recall at k

In [9]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    recommended_list = recommended_list[:k]
    prices_recommended = prices_recommended[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    money_recall_at_k = (np.inner(flags, prices_recommended)) / prices_bought.sum()
    
    return money_recall_at_k

In [10]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.5569620253164557

### Map k

In [11]:
# Здравствуйте. Немного запуталась с этой метрикой.
# Я рассчитала ap_k как в примере на Habr, для меня формула на Habr оказалась понятнее.
# То есть я поделила на количество рекомендованных товаров, а не на количество релевантных среди рекомендованных.
# Может быть, я что-то неправильно поняла.

In [12]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision_at_k = flags.sum() / len(recommended_list)
    
    return precision_at_k

In [13]:
precision_at_k(recommended_list, bought_list, k=5)

0.4

In [14]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1):
        
        if flags[i-1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / k
    
    return result

In [15]:
ap_k(recommended_list, bought_list, k=5)

0.3

In [16]:
users = [
    {'user_id': 1, 'recommended_list': [143, 156, 1134, 991, 27, 332, 43], 'bought_list': [521, 32, 143, 991]},
    {'user_id': 2, 'recommended_list': [1134, 27, 533, 11, 43, 59, 18], 'bought_list': [43, 34]},
    {'user_id': 3, 'recommended_list': [991, 1543, 15, 11, 43, 55, 23], 'bought_list': [991, 24, 10]}
]

In [17]:
def map_k(recommended_list, bought_list, k=5):
    
    sum_ap_k = 0
    for user in users:
        
        recommended_list = user['recommended_list']
        recommended_list = np.array(recommended_list)
        recommended_list = recommended_list[:k]
        
        bought_list = user['bought_list']
        bought_list = np.array(bought_list)
    
        flags = np.isin(recommended_list, bought_list)
    
        if sum(flags) == 0:
            return 0
    
        sum_ = 0
        for i in range(1, k+1):
        
            if flags[i-1] == True:
                p_k = precision_at_k(recommended_list, bought_list, k=i)
                sum_ += p_k
            
            ap_k = sum_ / k
            
        sum_ap_k += ap_k
        
    result = sum_ap_k / len(users)
            
    return result

In [18]:
map_k(recommended_list, bought_list, k=5)

0.18000000000000002

### NDCG at k

In [19]:
def dcg_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    if flags[0] == True:
        sum_ += 1
        
    for i in range(2, k+1):
        
        if flags[i-1] == True:
            gain = 1 / np.log2(i)
            sum_ += gain
    
    result = sum_ / k
    
    return result       

In [20]:
dcg_k(recommended_list, bought_list, k=5)

0.3

In [21]:
def idcg_k(k=5):
    
    sum_ = 1
        
    for i in range(2, k+1):
        
        gain = 1 / np.log2(i)
        sum_ += gain
    
    result = sum_ / k
    
    return result   

In [22]:
idcg_k(k=5)

0.7123212623289701

In [23]:
def ndcg(recommended_list, bought_list, k=5):
    
    result = dcg_k(recommended_list, bought_list, k=5) / idcg_k(k=5)
    
    return result

In [24]:
ndcg(recommended_list, bought_list, k=5)

0.4211582832992166

### Reciprocal rank

In [25]:
users = [
    {'user_id': 1, 'recommended_list': [143, 156, 1134, 991, 27], 'bought_list': [27]},
    {'user_id': 2, 'recommended_list': [1134, 27, 533, 11, 43], 'bought_list': [43]},
    {'user_id': 3, 'recommended_list': [991, 1543, 15, 11, 43], 'bought_list': [11]}
]

In [26]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    
    sum_rank = 0
    
    for user in users:
        
        recommended_list = user['recommended_list']
        recommended_list = np.array(recommended_list)
        
        bought_list = user['bought_list']
        bought_list = np.array(bought_list)
    
        flags = np.isin(recommended_list, bought_list)
        
        for i in range(1, k+1):
        
            if flags[i-1] == True:
                rank = 1 / (i)
                sum_rank += rank
        
    result = sum_rank / len(users)
            
    return result

In [27]:
reciprocal_rank(recommended_list, bought_list)

0.21666666666666667